In [1]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
import sparknlp
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import regexp_replace
from sparknlp.training import CoNLL
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.functions import col,lit
import numpy as np
import json
import pyspark.sql.functions as F

spark = sparknlp.start()
print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
14,application_1669518846024_0015,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Spark NLP version:  4.2.3
Apache Spark version:  3.2.0-amzn-0

In [2]:
df = spark.read.load('s3://hfg5-2022-11-25/uk-headlines/ukrainian-news-headlines.jsonl', format='json')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+
|            headline|length|
+--------------------+------+
|У США два штати в...|    41|
|РФ, забравши укра...|    80|
|Фельдман задеклар...|    75|
|В Одесі помилка н...|    65|
|Українські вчені ...|    63|
|На Житомирщині ст...|    58|
|НАЗК оприлюднило ...|    46|
|Пожежу у Зоні від...|    37|
|Міністром торгівл...|    68|
|Київська влада пр...|    53|
|Прикордонники: У ...|    68|
|Трамп у пошуку ви...|    60|
|Голубовська розпо...|    67|
|До України прибул...|    50|
|Майже 4 тис. укра...|    79|
|Бойовики обстріля...|    47|
|У Чорнобильський ...|    50|
|АКЦІЯ "УКРАЇНА БЕ...|    25|
|Кабмін доручив ти...|    61|
|В Україні запусти...|    51|
+--------------------+------+
only showing top 20 rows

In [4]:
documentAssembler = DocumentAssembler() \
     .setInputCol("headline")\
     .setOutputCol("document")

sentenceDetector = SentenceDetectorDLApproach() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence") \

tokenizer = Tokenizer() \
    .setInputCols("sentence") \
    .setOutputCol("token")

tokenClassifier = XlmRoBertaForTokenClassification.pretrained("xlmroberta_ner_uk_ner","uk") \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("ner")

pipeline = Pipeline(stages=[documentAssembler, sentenceDetector, tokenizer, tokenClassifier])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

xlmroberta_ner_uk_ner download started this may take some time.
Approximate size to download 387.4 MB
[OK!]

In [7]:
ner_model = pipeline.fit(df)

VBox()

An error was encountered:
Session 14 unexpectedly reached final status 'dead'. See logs:
stdout: 
:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml

stderr: 
Ivy Default Cache set to: /var/lib/livy/.ivy2/cache
The jars for the packages stored in: /var/lib/livy/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b92ef5d7-9e64-47ab-83ce-0a971593c576;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.2.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#pro

In [6]:
ner_model.show()

VBox()

An error was encountered:
Session 14 unexpectedly reached final status 'dead'. See logs:
stdout: 
:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml

stderr: 
Ivy Default Cache set to: /var/lib/livy/.ivy2/cache
The jars for the packages stored in: /var/lib/livy/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b92ef5d7-9e64-47ab-83ce-0a971593c576;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.2.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#pro